In [4]:
import pandas as pd
import json
import re

# Charger le DataFrame
df = pd.read_csv(r"../data/cleaned_tweets_with_sentiment.csv")

# Fonction pour extraire les valeurs des clés "Inconfort", "Sentiment", "Urgence"
def extract_json_values(json_str):
    try:
        # Isoler la partie JSON avant "Explication" ou autre texte ajouté
        json_part = re.search(r'\{.*?\}', json_str, re.S).group(0)

        # Nettoyer et parser le JSON
        cleaned_str = json_part.replace('\r\n', '').replace("'", '"')
        json_dict = json.loads(cleaned_str)

        # Renvoyer les valeurs des clés
        return json_dict.get("Inconfort"), json_dict.get("Sentiment"), json_dict.get("Urgence")
    except (json.JSONDecodeError, TypeError, AttributeError):
        return None, None, None

# Appliquer l'extraction sur la colonne 'sentiment'
df['Inconfort'], df['Sentiment'], df['Urgence'] = zip(*df['sentiment'].apply(extract_json_values))

# Supprimer l'ancienne colonne 'sentiment'
df.drop(columns=['sentiment', 'hour'], inplace=True)

# Fonction pour séparer la date et l'heure
def split_date_time(datetime_str):
    try:
        date_part, time_part = datetime_str.split(' ')
        time_part = time_part.replace('+00:00', '')  # Supprimer le décalage horaire
        return date_part, time_part
    except Exception:
        return None, None

# Appliquer la séparation sur la colonne 'date'
df['date'], df['Hours'] = zip(*df['date'].apply(split_date_time))

# Convertir les colonnes au bon format
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convertir en datetime
df['Hours'] = pd.to_datetime(df['Hours'], format='%H:%M:%S', errors='coerce').dt.time  # Convertir en time

# Convertir 'user_id' en chaîne de caractères et enlever les zéros inutiles
df['user_id'] = df['user_id'].apply(lambda x: str(int(x)) if pd.notna(x) else None)

# Afficher un aperçu du DataFrame
df.head()


,user_id,screen_name,name,date,tweet_text,tweet_length,criticite,Inconfort,Sentiment,Urgence,Hours
0,189684000000000011534336,lescausesjustes,AInnovation,2025-03-04,"@ENGIEgroup @ENGIEpartFR pour l'amour du ciel,...",113,1,90.0,Négatif,True,08:09:58
1,18967000000000000262144,Leoletonneau,Tigre & Dragon 🐯 🐉,2025-03-03,"@ENGIEpartFR vraiment des escrocs, des montant...",131,1,100.0,Négatif,True,23:25:45
2,189662999999999998164992,SouareFirst,So First,2025-03-03,@ENGIEpartFR bonjour engie les voleurs ! Atten...,184,1,100.0,Négatif,True,18:47:06
3,189661999999999987941376,julien_ducerf,Julien Ducerf,2025-03-03,@ENGIEpartFR c'est du harcèlement pour augment...,167,2,85.0,Négatif,False,17:50:42
4,189658000000000014155776,djofthp,jojodamido,2025-03-03,@ENGIEpartFR on vous parle,26,1,20.0,Neutre,False,15:24:10


In [ ]:
import pandas as pd
import time
from mistralai import Mistral

# Initialisation du client avec la clé API
client = Mistral(api_key="W028Ch2xLau3I03ivOHxSe4zUOt9BZ1d")

# Filtrer les tweets négatifs
df_negatifs = df[df['Sentiment'] == 'Négatif'].copy()

# Supprimer les tweets vides ou NaN
df_negatifs = df_negatifs.dropna(subset=['tweet_text'])
df_negatifs = df_negatifs[df_negatifs['tweet_text'].str.strip() != ""]

# Fonction pour analyser chaque tweet avec gestion des erreurs et du quota
def analyse_tweet_with_retry(tweet, retries=3, delay=5):
    attempt = 0
    while attempt < retries:
        try:
            print(f"📨 Envoi du tweet à l'API : {tweet[:50]}...")  # Debug
            response = client.agents.complete(
                agent_id="ag:b909df0a:20250310:untitled-agent:d426e319",  
                messages=[{"role": "user", "content": tweet}]
            )
            print(f"✅ Réponse reçue : {response.choices[0].message.content.strip()}")  # Debug
            return response.choices[0].message.content.strip()
        except Exception as e:
            if "429" in str(e):  # Gestion du quota
                print(f"⏳ Rate limit atteint, attente de {delay} sec...")
                time.sleep(delay)
                attempt += 1
            else:
                print(f"❌ Erreur API : {e}")
                return None  # Retourne None en cas d'échec

    return None  # Si toutes les tentatives échouent

# Appliquer l'analyse sur les tweets négatifs avec un délai
def apply_analysis_with_delay_and_retry(row):
    result = analyse_tweet_with_retry(row['tweet_text'])
    time.sleep(5)  # Pour éviter d'envoyer trop de requêtes à l'API
    return result

# Appliquer la fonction sur le DataFrame
df_negatifs['sentiment_analysis'] = df_negatifs.apply(apply_analysis_with_delay_and_retry, axis=1)

# Fusionner les résultats dans le DataFrame original
df = df.merge(df_negatifs[['tweet_text', 'sentiment_analysis']], on='tweet_text', how='left')

print("🔄 Analyse terminée et fusion des résultats avec le DataFrame original.")


In [17]:
import re

def extract_reason(text):
    # Si le texte est manquant (NaN ou vide), retourne NaN
    if pd.isna(text):
        return text
    
    # 1. Chercher le motif entre guillemets (ex: "Déception totale")
    match = re.search(r'"(.*?)"', text)
    if match:
        return match.group(1)
    
    # 2. Chercher le motif entre astérisques (ex: **Service médiocre**)
    match = re.search(r'\*\*(.*?)\*\*', text)
    if match:
        return match.group(1)
    
    # 3. Si aucun motif trouvé, récupérer après le dernier ':' (si existe)
    if ':' in text:
        return text.split(':')[-1].strip()

    # 4. Sinon, retourner le texte brut (au cas où aucun motif ne s'applique)
    return text.strip()

# Appliquer la fonction d'extraction sur la colonne 'sentiment_analysis'
df["sentiment_analysis"] = df["sentiment_analysis"].apply(extract_reason)


In [16]:
df.duplicated().sum()

np.int64(8)

In [10]:
df.to_csv(r"../data/cleaned_tweets_with_sentiment_updated.csv", index=False)